In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
print("Starting FileLoading")

#Load data
df = pd.read_csv('big.csv')
df.head()

Starting FileLoading


,CSCS,transmission,phase
0,"r-0.7-(0.0,0),(0,0.0)/Si3N4=0.0:C(0,0,0.0)/Ag=1.1",0.996598,5.347343
1,"r-0.7-(0.0,0),(0,0.0)/Si3N4=0.0:C(0,0,0.0)/Ag=1.1",0.996598,5.347343
2,"r-0.7-(0.0,0),(0,0.0)/Si3N4=0.0:C(0,0,0.0)/Ag=1.1",0.996598,5.347343
3,"r-0.7-(0.0,0),(0,0.0)/Si3N4=0.0:C(0,0,0.0)/Ag=1.1",0.996598,5.347343
4,"r-0.7-(0.0,0),(0,0.0)/Si3N4=0.0:C(0,0,0.0)/Ag=1.1",0.996598,5.347343


In [3]:

#parsing info, deleting extra stuff
df['sim_mode'] = df.CSCS.str.split("-").str.get(0)
df['sim_wavelength'] = df.CSCS.str.split("-").str.get(1)
df['latice_basis'] = df.CSCS.str.split("-").str.get(2).str.split("/").str.get(0)
df['material']= df.CSCS.str.split("-").str.get(2).str.split("/").str.get(1).str.split('=').str.get(0)
df['material_thickness'] = df.CSCS.str.split("-").str.get(2).str.split("/").str.get(1).str.split('=').str.get(1).str.split(':').str.get(0)
df['patter_shape'] = test = df.CSCS.str.split("-").str.get(2).str.split("/").str.get(1).str.split('=').str.get(1).str.split(':').str.get(1).str.get(0)
Temp = df.CSCS.str.split("-").str.get(2).str.split("/").str.get(1).str.split('=').str.get(1).str.split(':').str.get(1).str[1::]
Temp = Temp.str.replace("(","").str.replace(")","")
df['pattern_positionX'] = Temp.str.split(',').str[0]
df['pattern_positionY'] = Temp.str.split(',').str[1]
df['radius'] = Temp.str.split(',').str[2]

In [4]:
df = df[['transmission','phase','sim_wavelength','material_thickness','radius','latice_basis']].copy()
#renaming columns
df.columns = ['A','phi','lambda','T','R','P'];

In [5]:
#splitting data into wavelength
wavelengths = set(df['lambda'])
sorted_wavelength = sorted(wavelengths)
data1 = df[df['lambda']==sorted_wavelength[0]].reset_index(drop = True) #0.46
data2 = df[df['lambda']==sorted_wavelength[1]].reset_index(drop = True) #0.54
data3 = df[df['lambda']==sorted_wavelength[2]].reset_index(drop = True) #0.7
data1.head()

,A,phi,lambda,T,R,P
0,0.989683,0.833726,0.46,0.0,0.0,"(0.0,0),(0,0.0)"
1,0.989683,0.833726,0.46,0.0,0.0,"(0.0,0),(0,0.0)"
2,0.989683,0.833726,0.46,0.0,0.0,"(0.0,0),(0,0.0)"
3,0.989683,0.833726,0.46,0.0,0.0,"(0.0,0),(0,0.0)"
4,0.989683,0.833726,0.46,0.0,0.0,"(0.0,0),(0,0.0)"


In [6]:
combined = pd.merge(data1, data2,  how='inner', on=['T','R','P'])
combined.rename(index=str,columns = {'A_x':'A_1','A_y':'A_2','phi_x':'phi_1','phi_y':'phi_2','lambda_x':'lambda_1','lambda_y':'lambda_2'},inplace = True)

combined = pd.merge(combined, data3,  how='inner', on=['T','R','P'])
combined.rename(index=str,columns = {'A':'A_3','phi':'phi_3','lambda':'lambda_3'},inplace = True)


In [7]:
combined.head()
combined['P'] = combined['P'].str.replace("(","").str.replace(")","")

combined['P_1']=combined['P'].str.split(',').str[0]
combined['P_2']=combined['P'].str.split(',').str[3]
combined.drop(columns='P',inplace = True)
combined.drop_duplicates(inplace=True)
combined.reset_index(drop=True,inplace = True)

In [16]:
np.random.seed(1003)
#remove 5% for TEST
msk = np.random.rand(len(combined)) < 0.95

train = combined[msk]

test = combined[~msk]

print(train.shape)
print(test.shape)
print(combined.shape)

test.head()


(59268, 13)
(3172, 13)
(62440, 13)


,A_1,phi_1,lambda_1,T,R,A_2,phi_2,lambda_2,A_3,phi_3,lambda_3,P_1,P_2
29,0.989683,2.297702,0.46,0.74358974359,0.0,0.991472,5.642583,0.54,0.996598,6.129865,0.7,0.0,0.0
54,0.989683,0.833726,0.46,0.0,0.00460223537147,0.991472,0.904835,0.54,0.996598,5.347343,0.7,0.025641025641,0.025641025641
74,0.989683,0.833726,0.46,0.0,0.0111768573307,0.991472,0.904835,0.54,0.996598,5.347343,0.7,0.025641025641,0.025641025641
75,0.989683,0.833726,0.46,0.0,0.0115055884287,0.991472,0.904835,0.54,0.996598,5.347343,0.7,0.025641025641,0.025641025641
76,0.989683,0.833726,0.46,0.0,0.0118343195266,0.991472,0.904835,0.54,0.996598,5.347343,0.7,0.025641025641,0.025641025641


In [21]:
tt = np.array([test['A_1'],test['A_2'],test['A_3'],test['phi_1'],test['phi_2'],test['phi_3']])
test.iloc[0]['A_1']

0.9896830679999999